# Exercise 3: Undesirable Effect Relations in Tables

The goal of this exercise is to create a simple rule script for extracting undesirable effect information from tables. Declare a new annotation type named "UndesirableEffect" with three features of the type Annotation. 
The first feature named "class" represents the symptom class of the undesirable effect. The second feature named "effect" represents a actual effect of the relation. the thirs feature named "frequency" represents the frequency of the effect.
The project contains some exemplary input documents. The first relation in example1.txt is: class: "Metabolism and nutrition disorders", effect: "diabetes mellitus", frequency: "very common".
Specify a set of rules as generic as possible that extract all undesirable effect relations.

In [ ]:
%inputDir input_table1
%displayMode RUTA_COLORING

In [ ]:
%inputDir input_table2
%displayMode RUTA_COLORING

In [ ]:
%inputDir input_table1
%displayMode CSV
%csvConfig UndesirableEffect frequency class 
%writescript ./UndesirableEffect.ruta
%saveTypeSystem ./UndesirableEffectTypeSystem.xml

TYPESYSTEM org.apache.uima.ruta.engine.HtmlTypeSystem;
UIMAFIT org.apache.uima.ruta.engine.HtmlAnnotator;

EXEC(HtmlAnnotator, {TAG});
RETAINTYPE(WS, MARKUP);
TAG{-> TRIM(WS, MARKUP)};
RETAINTYPE;

DECLARE UndesirableEffect (Annotation class, Annotation effect, Annotation frequency);
ACTION UE(ANNOTATION class, ANNOTATION effect, ANNOTATION frequency) = 
    CREATE(UndesirableEffect, "class"= class, "effect" = effect, "frequency" = frequency) ;

DECLARE FrequencyInd;
WORDLIST FrequencyList = 'Frequencies.txt';
MARKFAST(FrequencyInd, FrequencyList, true);
FrequencyInd->{ANY f:FrequencyInd{-> UNMARK(f)};};

INT index;
DECLARE Cell(INT column);
DECLARE FirstRow, FirstColumn, FrequencyCell;
TR{STARTSWITH(TABLE)-> FirstRow};
TD{STARTSWITH(TR)-> FirstColumn};
TD{CONTAINS(FrequencyInd)-> FrequencyCell};
TR{->index=0}->{
    TD{->index = index+1, CREATE(Cell,"column"=index)};
};

DECLARE Chunk;
TD{-CONTAINS(FrequencyInd), -PARTOF(FirstColumn), -REGEXP("-") -> Chunk};
Chunk{CONTAINS(COMMA)-> SPLIT(COMMA)};

DECLARE Header;
"System organ class"-> Header;

c:TD{PARTOF(FirstColumn),-PARTOF(Header), -PARTOF(FrequencyCell)} 
    # f:FrequencyCell 
    # e:@Chunk{-PARTOF(UndesirableEffect) -> UE(c,e,f)};

// a rule for format 2
fc:Cell{PARTOF(FirstRow),PARTOF(FrequencyCell),fc.column==c.column}
    # cc:Cell{PARTOF(FirstColumn), -PARTOF(FrequencyCell)}
    # c:@Cell{CONTAINS(Chunk),-PARTOF(UndesirableEffect)}
    ->{e:@Chunk{-PARTOF(UndesirableEffect)-> UE(cc,e,fc)};};


In [ ]:
%inputDir input_table2
%displayMode CSV
%csvConfig UndesirableEffect frequency class 

SCRIPT UndesirableEffect;
CALL(UndesirableEffect);